In [1]:
from ultralytics import YOLO
import os
import random
import cv2
import numpy as np
import shutil
import supervision as sv
import cv2
import matplotlib.pyplot as plt


/home/mbergst/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [10]:
keypoints = {1: (44, 30), 
             2: (457, 30), 
             3: (44, 563), 
             4: (457, 563), 
             5: (44, 138), 
             6: (174, 138), 
             7: (174, 230), 
             8: (174, 346), 
             9: (174, 453), 
             10: (44, 453), 
             11: (44, 230), 
             12: (86, 230), 
             13: (86, 368), 
             14: (44, 368), 
             15: (130, 295),
             16: (457, 230), 
             17: (457, 295), 
             18: (457, 368), 
             19: (201, 295), 
             20: (387, 295), 
             21: (530, 295), 
             22: (870, 30), 
             23: (870, 563), 
             24: (870, 138), 
             25: (740, 138), 
             26: (740, 230), 
             27: (740, 346), 
             28: (740, 453), 29: (870, 453), 30: (870, 230), 31: (827, 230), 32: (827, 368), 33: (870, 368), 34: (785, 295), 35: (712, 295), 36: (601, 121)}

In [15]:
model = YOLO('yolov8n-pose.pt')

results = model.train(data='keypoint.yaml', epochs=3, batch=8, imgsz=(1920, 1080), project='/work/mbergst/TDT4265_Project/runs/keypoints')

New https://pypi.org/project/ultralytics/8.1.47 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24188MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=keypoint.yaml, epochs=3, time=None, patience=100, batch=8, imgsz=(1920, 1080), save=True, save_period=-1, cache=False, device=None, workers=8, project=/work/mbergst/TDT4265_Project/runs/keypoints, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, r

train: Scanning /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/labels/train.cache... 1441 images, 0 backgrounds, 1441 corrupt: 100%|██████████| 1441/1441 [00:00<?, ?it/s]

train: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/images/train/000001.jpg: ignoring corrupt image/label: labels require 77 columns each
train: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/images/train/000003.jpg: ignoring corrupt image/label: labels require 77 columns each
train: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/images/train/000004.jpg: ignoring corrupt image/label: labels require 77 columns each
train: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/images/train/000006.jpg: ignoring corrupt image/label: labels require 77 columns each
train: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/images/train/000008.jpg: ignoring corrupt image/label: labels require 77 col

ValueError: not enough values to unpack (expected 3, got 0)